In [12]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline


Build the Neural Network
========================

Neural networks comprise of layers/modules that perform operations on
data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace
provides all the building blocks you need to build your own neural
network. Every module in PyTorch subclasses the
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules
(layers). This nested structure allows for building and managing complex
architectures easily.

In the following sections, we\'ll build a neural network to classify
images in the FashionMNIST dataset.


In [13]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training
=======================

We want to be able to train our model on a hardware accelerator like the
GPU or MPS, if available. Let\'s check to see if
[torch.cuda](https://pytorch.org/docs/stable/notes/cuda.html) or
[torch.backends.mps](https://pytorch.org/docs/stable/notes/mps.html) are
available, otherwise we use the CPU.


In [14]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


Define the Class
================

We define our neural network by subclassing `nn.Module`, and initialize
the neural network layers in `__init__`. Every `nn.Module` subclass
implements the operations on input data in the `forward` method.


In [15]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, and move it to the `device`,
and print its structure.


In [16]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model\'s
`forward`, along with some [background
operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0
corresponding to each output of 10 raw predicted values for each class,
and dim=1 corresponding to the individual values of each output. We get
the prediction probabilities by passing it through an instance of the
`nn.Softmax` module.


In [17]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([9])


------------------------------------------------------------------------


Model Layers
============

Let\'s break down the layers in the FashionMNIST model. To illustrate
it, we will take a sample minibatch of 3 images of size 28x28 and see
what happens to it as we pass it through the network.


In [18]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten
==========

We initialize the
[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
layer to convert each 2D 28x28 image into a contiguous array of 784
pixel values ( the minibatch dimension (at dim=0) is maintained).


In [19]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear
=========

The [linear
layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using its
stored weights and biases.


In [20]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


nn.ReLU
=======

Non-linear activations are what create the complex mappings between the
model\'s inputs and outputs. They are applied after linear
transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use
[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
between our linear layers, but there\'s other activations to introduce
non-linearity in your model.


In [21]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.1778, -0.6358,  0.4384, -0.2247, -0.1924,  0.1068,  0.2005, -0.4070,
          0.2301,  0.1540, -0.0824, -0.3284,  0.0189,  0.5717,  0.3207,  0.3799,
          0.0042,  0.2600, -0.2829, -0.2648],
        [-0.1410, -0.6617,  0.2962, -0.3799, -0.4775,  0.0064,  0.3364, -0.2390,
          0.2942,  0.2815, -0.1095,  0.0507,  0.1896,  0.7977,  0.2641,  0.3419,
         -0.3654, -0.0095, -0.1602, -0.2606],
        [-0.2147, -0.8302,  0.3680, -0.3615, -0.1353, -0.0475,  0.3146, -0.2634,
          0.0242,  0.0405, -0.2964, -0.1679,  0.0510,  0.4761, -0.0191,  0.2266,
         -0.1938,  0.3007, -0.0447, -0.3648]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.4384, 0.0000, 0.0000, 0.1068, 0.2005, 0.0000, 0.2301,
         0.1540, 0.0000, 0.0000, 0.0189, 0.5717, 0.3207, 0.3799, 0.0042, 0.2600,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.2962, 0.0000, 0.0000, 0.0064, 0.3364, 0.0000, 0.2942,
         0.2815, 0.0000, 0.0507, 0.1896, 0.7977, 0.26

nn.Sequential
=============

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)
is an ordered container of modules. The data is passed through all the
modules in the same order as defined. You can use sequential containers
to put together a quick network like `seq_modules`.


In [22]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

nn.Softmax
==========

The last linear layer of the neural network returns [logits]{.title-ref}
- raw values in \[-infty, infty\] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)
module. The logits are scaled to values \[0, 1\] representing the
model\'s predicted probabilities for each class. `dim` parameter
indicates the dimension along which the values must sum to 1.


In [23]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

Model Parameters
================

Many layers inside a neural network are *parameterized*, i.e. have
associated weights and biases that are optimized during training.
Subclassing `nn.Module` automatically tracks all fields defined inside
your model object, and makes all parameters accessible using your
model\'s `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and
a preview of its values.


In [24]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0241,  0.0121, -0.0165,  ...,  0.0003,  0.0079, -0.0121],
        [ 0.0058, -0.0234, -0.0322,  ...,  0.0357,  0.0020, -0.0230]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0035,  0.0202], grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0184, -0.0243, -0.0395,  ...,  0.0239,  0.0160,  0.0313],
        [ 0.0346,  0.0158,  0.0216,  ...,  0.0090, -0.0340, -0.0233]],
       grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | 

------------------------------------------------------------------------


Further Reading
===============

-   [torch.nn API](https://pytorch.org/docs/stable/nn.html)


## Module

It is basically a base class foe all neural network models

In [47]:
import torch.nn as nn
""" This class can store layers and different models like sequential and also hold more atrributes
and methods like paramters, submodules and Forward method
It supports Automatic Differentiation which is automatic computation of gradients and can switch between devices for better compution"""
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.FlattenFunction=nn.Flatten()
    self.linear1=nn.Linear(16*16,128)
    self.relu1=nn.ReLU()
  def forward(self,x):
    x= self.FlattenFuncrion(x)
    x=self.linear1(x)
    x=self.relu1(x)
    return x


## Different Activation Functions

In [ ]:
relu=nn.Relu()
tanh=nn.Tanh()
sigmoid=nn.Sigmoid()
m = nn.LeakyReLU(0.1)
m2 = nn.ELU() # is value of Alpha not provided 1 is set by default
SM=nn.Softmax()(dim=1)

##Linear Layers

nn.Identity is a simple identity module that is part of the torch.nn module. It does not perform any computation on the input and simply returns it unchanged.

In [ ]:
class Model(nn.Module):
  def __init__(self):
    super().__init__()
    self.FlattenFunction=nn.Flatten()
    self.linear1=nn.Linear(16*16,128)
    self.relu1=nn.ReLU()
    self.identity = nn.Identity()  # Identity layer (giving any parameters wont make any changes)
  def forward(self,x):
    residual = x
    x= self.FlattenFuncrion(x)
    x=self.linear1(x)
    x=self.relu1(x)
    x = self.identity(x) + residual  # Add original input (residual) to output
    x = self.relu(x)


    return x


torch.nn.Linear(in_features, out_features, bias=True, device=None, dtype=None)